
## Notebook made by  

|** Name** | **Student id** | **email**|
|:- |:-|:-|
|Dylan Wijman |10651012|dylanwijman@hotmail.com|
|Eline Steensma|10589813|elinesteensma@gmail.com|
|Sjoerd Paardekooper|10278397|sjoerd.paardekooper@gmail.com|

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )


Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. 

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='https://github.com/dylanwijman/data-science/blob/master/Afbeeldingen/selfies.jpg?raw=true'/>


### Note
* **Assignments without the selfies or completely filled in information will not be graded and receive 0 points.**


# Part 1: obtaining information from the web

### RSS parsing

Make a notebook that performs the following steps.

1. Create a script that retrieves all urls of rss feeds from <http://www.volkskrant.nl/rss/feeds/>. Use urllib2 and beautifulsoup for this. Store the urls in a list.
    * **update 2016**
    * As all Dutch sites, Volkskrant asks whether you accept cookies. This makes simple collecting webpages a lot harder. 
    * The code in the code cell below does the trick. 
    * After running this, I could collect further files from Volkskrant without additional cookie hassle.
2. Download all rss feeds and store them on disk.
3. Parse all RSS feeds using `lxml`. Create a list of  dicts with fields "channel", "url", "title", "date" in which you store this information for each item.
4. Compute some statistics about this dict: how many items, how many per channel, are there doubles (items occuring in several channels), etc.
5. Write this list as a csv file, store on disk, and upload to Google fusion tables.
6. Download all articles (once), parse out the text and store as pairs (date,text) in a list.
7. Count per day the number of words, and the number of unique words. Show this in a plot.

In [11]:
import cookielib # Thanks to http://stackoverflow.com/questions/29395407/enabling-cookies-with-urllib
import urllib2
import urllib
import json
import requests
import os
from bs4 import BeautifulSoup
import re 
import feedparser
from lxml import etree
from lxml import objectify
from itertools import chain
import csv

url = 'https://rawgit.com/maartensukel/Data-Science/master/deVolkskrant.html'

# with urllib2 and handling cookies
cookiejar= cookielib.LWPCookieJar()
opener= urllib2.build_opener( urllib2.HTTPCookieProcessor(cookiejar) )
response=opener.open(url)
html_doc= ' '.join(response.readlines())
 
rsssoup = BeautifulSoup(html_doc, "lxml")

In [2]:
from collections import Counter

#Maak een lijst met alle urls van de RSSfeed van de volkskrant
feeds = [link.get('href') for link in rsssoup.findAll(href=re.compile("\.xml$"))]

rssData = []
tags = ["channel", "url", "title", "date"]
# Loop door de RSS feed links en sla de xml lokaal op
for link in feeds:
    response=opener.open(link)
    xml_doc= ' '.join(response.readlines())
    xmlsoup = BeautifulSoup(xml_doc, "lxml")
    name = xmlsoup.title.string+'.html'
    with open(name, 'w') as f:
        f.write(xmlsoup.encode())
    f.close()
    # Maak een dictionary met keys: channel, url, title en date aan per link in de feed
    # Dict comprehension voor 1.3
    rssData.append({tag: xmlsoup.find_all(tag) for tag in tags})

In [16]:
information = []

rss = urllib2.urlopen(link)
parsed = etree.parse(rss)
root = parsed.getroot()

channel = parsed.findtext('//title')
urls = [ url.text for url in parsed.findall('//item//link')]
titlestrings = [ t.text for t in parsed.xpath('//item//title')]
dates = [ d.text for d in parsed.xpath('//item//pubDate')]
    
for i in feeds:
    urlsea = urls.pop()
    titlestringsea = titlestrings.pop()
    datesea = dates.pop()
    info = {
    'CHANNEL:': channel,
    'URL:': urlsea,
    'TITLE:': titlestringsea,
    'DATE:': datesea,
    }
    information.append(info)

print information

[{'CHANNEL:': 'VK:Voordeel', 'TITLE:': "'Nederlandse vliegtuigen dit jaar vaker vertraagd'", 'DATE:': 'Wed, 17 Sep 2014 09:14:00 GMT', 'URL:': 'http://www.volkskrant.nl/economie/-nederlandse-vliegtuigen-dit-jaar-vaker-vertraagd~a3749493/'}, {'CHANNEL:': 'VK:Voordeel', 'TITLE:': 'Insectenburgers binnenkort in de supermarkt?', 'DATE:': 'Wed, 17 Sep 2014 12:53:00 GMT', 'URL:': 'http://www.volkskrant.nl/buitenland/insectenburgers-binnenkort-in-de-supermarkt~a3749614/'}, {'CHANNEL:': 'VK:Voordeel', 'TITLE:': 'Pensioenfondsen gaan hypotheken verstrekken', 'DATE:': 'Fri, 19 Sep 2014 18:50:00 GMT', 'URL:': 'http://www.volkskrant.nl/voordeel/pensioenfondsen-gaan-hypotheken-verstrekken~a3751247/'}, {'CHANNEL:': 'VK:Voordeel', 'TITLE:': 'Veel jongeren hebben nu al een betalingsachterstand', 'DATE:': 'Tue, 23 Sep 2014 04:37:00 GMT', 'URL:': 'http://www.volkskrant.nl/binnenland/veel-jongeren-hebben-nu-al-een-betalingsachterstand~a3752960/'}, {'CHANNEL:': 'VK:Voordeel', 'TITLE:': 'Mogelijk lucht voo

In [7]:
# 1.4
# Channel is een CSS tag: <channel>. Komt een keer voor per feed vandaar.
# a. Lijst met het aantal berichten per RSS feed
lenItemsPerFeed = [len(channel.find_all('item')) for channel in [feed['channel'][0] for feed in rssData]]
print "a:", lenItemsPerFeed
# b. Amount of items per feed
print "b:", sum(lenItemsPerFeed)
# c. Average amount of items
print "c:", sum(lenItemsPerFeed)/len(lenItemsPerFeed)
# d. Zijn er dubbels?
itemsPerFeed = [channel.find_all('item') for channel in [feed['channel'][0] for feed in rssData]]
# Maak 1 lijst met alle items
items = list(chain.from_iterable(itemsPerFeed))
# Geeft lijst met een count van items die meer dan een keer voorkomen en len geeft het aantal items weer die dubbel voorkomen
# http://stackoverflow.com/questions/9835762/find-and-list-duplicates-in-python-list
# Hieronder is gecomment door lange wachttijd
print "d:", len([count for item, count in Counter(items).items() if count > 1])

a: [40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 9, 40, 40, 0, 0, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40]
b: 1169
c: 36
d: 191


In [17]:
# 1.5
with open('eenpuntvijf.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(information[1].keys())
    for row in information:
        writer.writerow(row.values())

UnicodeEncodeError: 'ascii' codec can't encode character u'\xeb' in position 5: ordinal not in range(128)

### JSON parsing

1. Download <http://maartenmarx.nl/teaching/ISatWork/NoteBooks/consuming-json-data-from-a-web-service.ipynb>, remove all code blocks, and turn it into a notebook again. 
2. Check that what you did is correct and you did not remove too much using a notebook viewer.
3. Now extract all code from the downloaded notebook, save it to a file, and execute it as a Python script. Does it give errors? Is it syntactically correct?

In [ ]:
#import notebook
url = "http://maartenmarx.nl/teaching/ISatWork/NoteBooks/consuming-json-data-from-a-web-service.ipynb"
jsonfile= urllib2.urlopen(url)
jsonfile = jsonfile.read()  # The jsonfile as a Python string
json_as_python_object = json.loads(jsonfile) # The josnfile transformed into a Python dict
cells = json_as_python_object['worksheets'][0]['cells']
codeblocks = [cell['input'] for cell in cells if str(cell['cell_type']) == 'code']
f = open("output.py", "w")
for x in codeblocks:
    for y in x:
      f.write(str(y))
    f.write("\n")
f.close()

### PDF parsing

1. Save a wordfile as PDF, open it in Python, extract all text. Describe the differences, if any. Try the same with a two column PDF file from the web. This exercise gets more interesting if you use _difficult_ PDF. Why not try <http://wch.github.io/latexsheet/latexsheet.pdf>?

* Is the word order still as it should be?
* What about the strange characters?

In [ ]:
!curl http://wch.github.io/latexsheet/latexsheet.pdf > latex.pdf; pdftotext  latex.pdf
!pdftotext latex.pdf

In [ ]:
!curl http://www.koopman-assurantien.nl/images1/%28118%29%2008.08.pdf > koopman.pdf; pdftotext koopman.pdf
!pdftotext koopman.pdf

In [ ]:
!curl https://github.com/dylanwijman/data-science/blob/master/Documenten/grondwet-voor-het-koningrijk-der-nederlanden-1815.pdf > grondwet.pdf; pdftotext grondwet.pdf
!pdftotext grondwet.pdf

Word document
https://github.com/dylanwijman/data-science/blob/master/Documenten/grondwet-voor-het-koningrijk-der-nederlanden-1815.doc

PDF document
https://github.com/dylanwijman/data-science/blob/master/Documenten/grondwet-voor-het-koningrijk-der-nederlanden-1815.pdf

1. Verschillen tussen .doc en .pdf text extracting files

1.2 Try the same with a two column PDF file from the web.

Bij de 'Koopman' PDF file met twee kolommen kan de file worden extract zonder het behouden van de layout (-layout). Op deze manier wordt er niet in de leesrichting gezocht naar tekens, van links naar rechts. Op deze manier worden de kolommen tekst bijeen gehouden. Enkele koppen in het 'Koopman' bestand zijn wel verkeerd geplaatst, in plaats van onder elkaar wordt de leesrichting hier nog wel aangehouden als koppen op dezelfde horizontale regel staan.

This exercise gets more interesting if you use difficult PDF
2 Is the word order still as it should be?

In het Latex bestand worden de kolommen in kolommen niet goed onderscheiden als tekst die naast elkaar staat, ook niet met -layout. De leesvolgorde gaat na Packages naar Lists en daarna ineens naar de derde kolom naar Justification, waaronder alle informatie van Lists staat.

3 What about the strange characters?

De accenten zijn niet allemaal goed weergeven en vaak uit elkaar gehaald. Een Ç (c cedille) wordt niet als 1 teken weergeven maar een losse , (komma) en en c: , c.

# Part 2 Python recap

Download [PythonRecap2.0.ipynb](PythonRecap2.0.ipynb),  and answer all questions as asked.